In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
        SparkSession.builder
        .appName("Test_task_3")
        .getOrCreate()
        )

file_path_actors = ("/home/ilya/github/"
                   "DE_course_repo/python_projects/"
                   "PySpark/data/films_actors/actors.csv")

file_path_movie_actors = ("/home/ilya/github/"
                          "DE_course_repo/python_projects/"
                          "PySpark/data/films_actors/movie_actors.csv")

file_path_movies = ("/home/ilya/github/"
                          "DE_course_repo/python_projects/"
                          "PySpark/data/films_actors/movies.csv")                 

actors_df = spark.read.csv(file_path_actors,
                            header=True,
                            inferSchema=True)

movie_actors_df = spark.read.csv(file_path_movie_actors,
                            header=True,
                            inferSchema=True)

movies_df = spark.read.csv(file_path_movies,
                            header=True,
                            inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/16 18:19:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/16 18:19:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
actors_df.createOrReplaceTempView("actors")
movie_actors_df.createOrReplaceTempView("movie_actors")
movies_df.createOrReplaceTempView("movies")

In [4]:
df = spark.sql("""SELECT * FROM movie_actors ma
                  LEFT JOIN actors a
                  on ma.actor_id = a.actor_id
                  LEFT JOIN movies m
                  on ma.movie_id = m.movie_id""")

df.show()

+--------+--------+--------+--------+----------+---------+--------+--------+------+------------+-------------+
|movie_id|actor_id|actor_id|    name|birth_date|  country|movie_id|   title| genre|release_date|       budget|
+--------+--------+--------+--------+----------+---------+--------+--------+------+------------+-------------+
|       1|      25|      25|Actor_25|2008-12-31|    India|       1| Movie_1|Horror|  2000-12-31|8.660058311E7|
|      16|       5|       5| Actor_5|1968-12-31|    India|      16|Movie_16|Comedy|  2015-12-31|6.098669335E7|
|       6|      16|      16|Actor_16|1990-12-31|    India|       6| Movie_6|Action|  2005-12-31|1.476121831E7|
|      16|      11|      11|Actor_11|1980-12-31|      USA|      16|Movie_16|Comedy|  2015-12-31|6.098669335E7|
|      14|      21|      21|Actor_21|2000-12-31|   Canada|      14|Movie_14|Sci-Fi|  2013-12-31|8.049514883E7|
|       3|       6|       6| Actor_6|1970-12-31|      USA|       3| Movie_3|Action|  2002-12-31| 1.80157747E7|
|

In [7]:
spark.sql("""SELECT genre, count(movie_id) as top_5_genre
             FROM movies
             GROUP BY genre
             ORDER BY top_5_genre DESC
             LIMIT 5""").show()

+------+-----------+
| genre|top_5_genre|
+------+-----------+
| Drama|          6|
|Action|          6|
|Comedy|          4|
|Horror|          2|
|Sci-Fi|          2|
+------+-----------+



In [10]:
spark.sql("""SELECT a.name, count(ma.movie_id) as num_movies
             FROM movie_actors ma
                  LEFT JOIN actors a
                  on ma.actor_id = a.actor_id
                  LEFT JOIN movies m
                  on ma.movie_id = m.movie_id
             GROUP BY a.name
             ORDER BY num_movies DESC
             LIMIT 1""").show()

+--------+----------+
|    name|num_movies|
+--------+----------+
|Actor_17|         5|
+--------+----------+



In [13]:
spark.sql("""SELECT genre, avg(budget) as avg_budget
             FROM movies
             GROUP BY genre
             --ORDER BY avg_budget DESc""").show()

+------+--------------------+
| genre|          avg_budget|
+------+--------------------+
| Drama| 6.076021856166667E7|
|Horror|      8.7281876775E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
|Sci-Fi|       7.809715175E7|
+------+--------------------+



In [21]:
spark.sql("""SELECT m.title, a.country, count(ma.actor_id) as num_actors
             FROM movie_actors ma
                  LEFT JOIN actors a
                  on ma.actor_id = a.actor_id
                  LEFT JOIN movies m
                  on ma.movie_id = m.movie_id
                  GROUP BY m.title, a.country
                  HAVING num_actors > 1""").show()

+--------+---------+----------+
|   title|  country|num_actors|
+--------+---------+----------+
| Movie_7|    India|         2|
| Movie_3|      USA|         2|
|Movie_10|       UK|         2|
|Movie_15|    India|         2|
|Movie_18|Australia|         2|
| Movie_1|    India|         3|
| Movie_2|      USA|         2|
| Movie_7|      USA|         2|
|Movie_10|      USA|         2|
+--------+---------+----------+



In [22]:
spark.stop()